In [4]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [5]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [6]:
item_props_df = spark.read \
    .option("header", "true") \
    .csv("file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/raw/item_properties*.csv")

item_props_df.printSchema()
item_props_df.show(5)



root
 |-- timestamp: string (nullable = true)
 |-- itemid: string (nullable = true)
 |-- property: string (nullable = true)
 |-- value: string (nullable = true)

+-------------+------+----------+--------------------+
|    timestamp|itemid|  property|               value|
+-------------+------+----------+--------------------+
|1435460400000|460429|categoryid|                1338|
|1441508400000|206783|       888|1116713 960601 n2...|
|1439089200000|395014|       400|n552.000 639502 n...|
|1431226800000| 59481|       790|          n15360.000|
|1431831600000|156781|       917|              828513|
+-------------+------+----------+--------------------+
only showing top 5 rows



In [7]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number

window_spec = Window \
    .partitionBy("itemid", "property") \
    .orderBy(col("timestamp").desc())

latest_item_props_df = item_props_df \
    .withColumn("rn", row_number().over(window_spec)) \
    .filter(col("rn") == 1) \
    .drop("rn")


In [8]:
dim_items_df = latest_item_props_df \
    .groupBy("itemid") \
    .pivot("property") \
    .agg({"value": "first"})

dim_items_df.printSchema()


root
 |-- itemid: string (nullable = true)
 |-- 0: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- 10: string (nullable = true)
 |-- 100: string (nullable = true)
 |-- 1000: string (nullable = true)
 |-- 1001: string (nullable = true)
 |-- 1002: string (nullable = true)
 |-- 1003: string (nullable = true)
 |-- 1004: string (nullable = true)
 |-- 1005: string (nullable = true)
 |-- 1006: string (nullable = true)
 |-- 1007: string (nullable = true)
 |-- 1008: string (nullable = true)
 |-- 1009: string (nullable = true)
 |-- 101: string (nullable = true)
 |-- 1010: string (nullable = true)
 |-- 1011: string (nullable = true)
 |-- 1012: string (nullable = true)
 |-- 1013: string (nullable = true)
 |-- 1014: string (nullable = true)
 |-- 1015: string (nullable = true)
 |-- 1016: string (nullable = true)
 |-- 1017: string (nullable = true)
 |-- 1018: string (nullable = true)
 |-- 1019: string (nullable = true)
 |-- 102: string (nullable = true)
 |-- 1020: string (nullable = tr

In [9]:
dim_items_df.write.mode("overwrite") \
    .parquet("file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/processed/dim_items")


In [10]:
category_df = spark.read \
    .option("header", "true") \
    .csv("file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/raw/category_tree.csv")

category_df.printSchema()
category_df.show(5)


root
 |-- categoryid: string (nullable = true)
 |-- parentid: string (nullable = true)

+----------+--------+
|categoryid|parentid|
+----------+--------+
|      1016|     213|
|       809|     169|
|       570|       9|
|      1691|     885|
|       536|    1691|
+----------+--------+
only showing top 5 rows



In [11]:
category_df.write.mode("overwrite") \
    .parquet("file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/processed/dim_category")
